# Gradient Boosting 6133 Тюгаев Никита Павлович

## 1. Считать в `pandas.DataFrame` датасет automobile из ucimlrepo

In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Импортируем модели градиентного бустинга
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
import catboost as cb
from catboost import CatBoostClassifier, CatBoostRegressor

# Импортируем SHAP для анализа важности признаков
import shap

# Импортируем библиотеку для визуализации деревьев решений
from sklearn.tree import export_graphviz
import graphviz

# Импортируем датасет из ucimlrepo
from ucimlrepo import fetch_ucirepo

# Настройка отображения графиков
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set(font_scale=1.2)

# Отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Загружаем датасет automobile из ucimlrepo
automobile = fetch_ucirepo(id=10)

# Получаем данные и метаданные
X = automobile.data.features
y = automobile.data.targets
metadata = automobile.metadata
variable_info = automobile.variables

# Объединяем признаки и целевую переменную в один датафрейм
df = pd.concat([X, y], axis=1)

# Выводим информацию о датасете
print(f"Название датасета: {metadata['name']}")
print(f"Количество наблюдений: {metadata['num_instances']}")
print(f"Количество признаков: {metadata['num_features']}")
print(f"Целевая переменная для регрессии: price")

# Выводим первые 5 строк датафрейма
df.head()

## 2. Датасет и подготовка данных

### 2.1 Привести описание датасета

In [ ]:
# Выводим описание датасета
print(metadata['abstract'])

# Выводим информацию о переменных
variable_info

In [ ]:
# Получаем общую информацию о датафрейме
df.info()

In [ ]:
# Статистическое описание числовых признаков
df.describe()

### 2.2 Осуществить предобработку данных (избавиться от `null`, убрать некоторые признаки и т.п.) - "подчистить данные"

In [ ]:
# Проверяем наличие пропущенных значений
print("Количество пропущенных значений в каждом столбце:")
print(df.isnull().sum())

# Проверяем наличие значений '?' (часто используется в UCI датасетах вместо NaN)
for col in df.columns:
    if df[col].dtype == 'object':
        if (df[col] == '?').any():
            print(f"Столбец {col} содержит значения '?': {(df[col] == '?').sum()}")

In [ ]:
# Заменяем значения '?' на NaN
df_clean = df.replace('?', np.nan)

# Преобразуем столбцы с числовыми данными из строкового типа в числовой
numeric_cols = ['normalized-losses', 'bore', 'stroke', 'horsepower', 'peak-rpm', 'price']
for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Проверяем наличие пропущенных значений после преобразования
print("Количество пропущенных значений после преобразования:")
print(df_clean.isnull().sum())

In [ ]:
# Заменяем пропущенные значения
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Для числовых признаков используем медиану
for col in numerical_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    
# Для категориальных признаков используем наиболее частое значение
for col in categorical_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])
    
print(f"Размер датафрейма после заполнения пропущенных значений: {df_clean.shape}")

# Удаляем целевые переменные из списка числовых признаков
if 'price' in numerical_cols:
    numerical_cols.remove('price')

print("Категориальные признаки:")
print(categorical_cols)
print("\nЧисловые признаки:")
print(numerical_cols)
print("\nЦелевая переменная для регрессии: price")

### 2.3 Нормализовать численные данные

In [ ]:
# Создаем копию датафрейма для нормализации
df_normalized = df_clean.copy()

# Создаем объект StandardScaler
scaler = StandardScaler()

# Нормализуем числовые признаки
df_normalized[numerical_cols] = scaler.fit_transform(df_clean[numerical_cols])

# Выводим первые 5 строк нормализованного датафрейма
df_normalized.head()

### 2.4 Разбить выборку на обучающую и тестовую

In [ ]:
# Создаем целевую переменную для классификации (symblings)
# Предположим, что мы будем использовать признак 'num-of-doors' в качестве целевой переменной для классификации
# Так как в датасете нет признака 'symblings', мы будем использовать 'num-of-doors' как аналог

# Кодируем категориальную целевую переменную
label_encoder = LabelEncoder()
df_clean['doors_encoded'] = label_encoder.fit_transform(df_clean['num-of-doors'])

# Сохраняем исходные датафреймы для дальнейшего использования
df_original = df_clean.copy()

# Разделяем данные на признаки (X) и целевые переменные (y_reg для регрессии, y_cls для классификации)
X = df_clean.drop(['price', 'doors_encoded', 'num-of-doors'], axis=1)
y_reg = df_clean['price']  # Целевая переменная для регрессии
y_cls = df_clean['doors_encoded']  # Целевая переменная для классификации

# Разбиваем выборку на обучающую и тестовую (80% обучающая, 20% тестовая)
X_train, X_test, y_reg_train, y_reg_test, y_cls_train, y_cls_test = train_test_split(
    X, y_reg, y_cls, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

#### Формирование наборов с закодированными категориальными признаками и без обработки

In [ ]:
# Создаем препроцессор для кодирования категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols)
    ])

# Применяем препроцессор к обучающей выборке
X_train_encoded = preprocessor.fit_transform(X_train)

# Применяем препроцессор к тестовой выборке
X_test_encoded = preprocessor.transform(X_test)

# Получаем имена признаков после кодирования
encoded_features = numerical_cols.copy()
for i, category in enumerate(categorical_cols):
    # Получаем категории для каждого категориального признака
    categories = preprocessor.transformers_[1][1].categories_[i]
    # Добавляем имена закодированных признаков (исключая первую категорию из-за drop='first')
    encoded_features.extend([f"{category}_{cat}" for cat in categories[1:]])

print(f"Размер обучающей выборки после кодирования: {X_train_encoded.shape}")
print(f"Размер тестовой выборки после кодирования: {X_test_encoded.shape}")
print(f"Количество признаков после кодирования: {len(encoded_features)}")

In [ ]:
# Сохраняем категориальные признаки для CatBoost и XGBoost
cat_features_indices = [X_train.columns.get_loc(col) for col in categorical_cols]
print(f"Индексы категориальных признаков: {cat_features_indices}")

## 3. Для каждого из классификаторов ($AdaBoost$, $GradientBoostingClassifier$, $XGBoost$, $CatBoost$)

### 3.1 $AdaBoost$

#### 3.1.1 С использованием `GridSearchCV` осуществить подбор гиперпараметра модели

In [ ]:
# Определяем сетку параметров для AdaBoost
adaboost_params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0],
    'algorithm': ['SAMME', 'SAMME.R']
}

# Создаем объект AdaBoostClassifier
adaboost = AdaBoostClassifier(random_state=42)

# Создаем объект GridSearchCV
adaboost_grid = GridSearchCV(adaboost, adaboost_params, cv=5, scoring='accuracy', n_jobs=-1)

# Обучаем модель с подбором параметров
adaboost_grid.fit(X_train_encoded, y_cls_train)

# Выводим лучшие параметры
print(f"Лучшие параметры для AdaBoost: {adaboost_grid.best_params_}")
print(f"Лучший score (accuracy): {adaboost_grid.best_score_:.4f}")

# Получаем лучшую модель
best_adaboost = adaboost_grid.best_estimator_

# Делаем предсказания на тестовой выборке
y_pred_adaboost = best_adaboost.predict(X_test_encoded)

#### 3.1.2 Вывести метрики на тестовом наборе

In [ ]:
# Функция для вычисления и вывода метрик классификации
def evaluate_classifier(y_true, y_pred, model_name):
    print(f"Метрики для модели {model_name}:")
    print(classification_report(y_true, y_pred))
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

# Вычисляем метрики для AdaBoost
adaboost_metrics = evaluate_classifier(y_cls_test, y_pred_adaboost, "AdaBoost")

### 3.2 $GradientBoostingClassifier$

#### 3.2.1 С использованием `GridSearchCV` осуществить подбор гиперпараметра модели

In [ ]:
# Определяем сетку параметров для GradientBoostingClassifier
gb_params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5]
}

# Создаем объект GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)

# Создаем объект GridSearchCV
gb_grid = GridSearchCV(gb, gb_params, cv=5, scoring='accuracy', n_jobs=-1)

# Обучаем модель с подбором параметров
gb_grid.fit(X_train_encoded, y_cls_train)

# Выводим лучшие параметры
print(f"Лучшие параметры для GradientBoostingClassifier: {gb_grid.best_params_}")
print(f"Лучший score (accuracy): {gb_grid.best_score_:.4f}")

# Получаем лучшую модель
best_gb = gb_grid.best_estimator_

# Делаем предсказания на тестовой выборке
y_pred_gb = best_gb.predict(X_test_encoded)

#### 3.2.2 Вывести метрики на тестовом наборе

In [ ]:
# Вычисляем метрики для GradientBoostingClassifier
gb_metrics = evaluate_classifier(y_cls_test, y_pred_gb, "GradientBoostingClassifier")

### 3.3 $XGBoost$

#### 3.3.1 С использованием `GridSearchCV` осуществить подбор гиперпараметра модели

In [ ]:
# Определяем сетку параметров для XGBoost
xgb_params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0, 0.1, 1.0],
    'reg_lambda': [0, 0.1, 1.0]
}

# Создаем объект XGBClassifier
xgb_cls = XGBClassifier(random_state=42, early_stopping_rounds=10, eval_metric='mlogloss')

# Создаем объект GridSearchCV
xgb_grid = GridSearchCV(xgb_cls, xgb_params, cv=5, scoring='accuracy', n_jobs=-1)

# Обучаем модель с подбором параметров
xgb_grid.fit(X_train_encoded, y_cls_train)

# Выводим лучшие параметры
print(f"Лучшие параметры для XGBoost: {xgb_grid.best_params_}")
print(f"Лучший score (accuracy): {xgb_grid.best_score_:.4f}")

# Получаем лучшую модель
best_xgb = xgb_grid.best_estimator_

# Делаем предсказания на тестовой выборке
y_pred_xgb = best_xgb.predict(X_test_encoded)

#### 3.3.2 Обучить модель XGBoost с найденными гиперпараметрами на обучающей выборке с категориальными признаками "как есть"

In [ ]:
# Создаем модель XGBoost с найденными гиперпараметрами
xgb_cat = XGBClassifier(
    n_estimators=best_xgb.n_estimators,
    learning_rate=best_xgb.learning_rate,
    max_depth=best_xgb.max_depth,
    subsample=best_xgb.subsample,
    colsample_bytree=best_xgb.colsample_bytree,
    reg_alpha=best_xgb.reg_alpha,
    reg_lambda=best_xgb.reg_lambda,
    random_state=42,
    enable_categorical=True  # Включаем поддержку категориальных признаков
)

# Обучаем модель на данных с категориальными признаками "как есть"
xgb_cat.fit(X_train, y_cls_train)

# Делаем предсказания на тестовой выборке
y_pred_xgb_cat = xgb_cat.predict(X_test)

#### 3.3.3 Вывести метрики на тестовом наборе

In [ ]:
# Вычисляем метрики для XGBoost с закодированными признаками
xgb_metrics = evaluate_classifier(y_cls_test, y_pred_xgb, "XGBoost (с закодированными признаками)")

# Вычисляем метрики для XGBoost с категориальными признаками "как есть"
xgb_cat_metrics = evaluate_classifier(y_cls_test, y_pred_xgb_cat, "XGBoost (с категориальными признаками 'как есть')")

### 3.4 $CatBoost$

#### 3.4.1 С использованием `grid_search` осуществить подбор гиперпараметра модели

In [ ]:
# Определяем сетку параметров для CatBoost
catboost_params = {
    'iterations': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5],
    'border_count': [32, 64],
    'bagging_temperature': [0, 1],
    'random_strength': [1, 10]
}

# Создаем объект CatBoostClassifier
catboost_cls = CatBoostClassifier(
    random_seed=42,
    verbose=False,
    train_size=0.8,  # Часть обучающей выборки пойдет в валидационный набор
    early_stopping_rounds=10
)

# Создаем объект GridSearchCV
catboost_grid = GridSearchCV(catboost_cls, catboost_params, cv=5, scoring='accuracy', n_jobs=-1)

# Обучаем модель с подбором параметров
catboost_grid.fit(X_train_encoded, y_cls_train)

# Выводим лучшие параметры
print(f"Лучшие параметры для CatBoost: {catboost_grid.best_params_}")
print(f"Лучший score (accuracy): {catboost_grid.best_score_:.4f}")

# Получаем лучшую модель
best_catboost = catboost_grid.best_estimator_

# Делаем предсказания на тестовой выборке
y_pred_catboost = best_catboost.predict(X_test_encoded)

#### 3.4.2 Обучить модель CatBoost с найденными гиперпараметрами на обучающей выборке с категориальными признаками "как есть"

In [ ]:
# Создаем модель CatBoost с найденными гиперпараметрами
catboost_cat = CatBoostClassifier(
    iterations=best_catboost.iterations,
    learning_rate=best_catboost.learning_rate,
    depth=best_catboost.depth,
    l2_leaf_reg=best_catboost.l2_leaf_reg,
    border_count=best_catboost.border_count,
    bagging_temperature=best_catboost.bagging_temperature,
    random_strength=best_catboost.random_strength,
    random_seed=42,
    verbose=False,
    cat_features=cat_features_indices  # Указываем индексы категориальных признаков
)

# Обучаем модель на данных с категориальными признаками "как есть"
catboost_cat.fit(X_train, y_cls_train)

# Делаем предсказания на тестовой выборке
y_pred_catboost_cat = catboost_cat.predict(X_test)

#### 3.4.3 Вывести метрики на тестовом наборе

In [ ]:
# Вычисляем метрики для CatBoost с закодированными признаками
catboost_metrics = evaluate_classifier(y_cls_test, y_pred_catboost, "CatBoost (с закодированными признаками)")

# Вычисляем метрики для CatBoost с категориальными признаками "как есть"
catboost_cat_metrics = evaluate_classifier(y_cls_test, y_pred_catboost_cat, "CatBoost (с категориальными признаками 'как есть')")

### 3.5 Сравнить модели, выбрать лучшую

In [ ]:
# Создаем датафрейм для сравнения метрик
metrics_comparison = pd.DataFrame({
    'Модель': ['AdaBoost', 'GradientBoostingClassifier', 'XGBoost (закодированные)', 'XGBoost (как есть)', 'CatBoost (закодированные)', 'CatBoost (как есть)'],
    'Accuracy': [adaboost_metrics['accuracy'], gb_metrics['accuracy'], xgb_metrics['accuracy'], xgb_cat_metrics['accuracy'], catboost_metrics['accuracy'], catboost_cat_metrics['accuracy']],
    'Precision': [adaboost_metrics['precision'], gb_metrics['precision'], xgb_metrics['precision'], xgb_cat_metrics['precision'], catboost_metrics['precision'], catboost_cat_metrics['precision']],
    'Recall': [adaboost_metrics['recall'], gb_metrics['recall'], xgb_metrics['recall'], xgb_cat_metrics['recall'], catboost_metrics['recall'], catboost_cat_metrics['recall']],
    'F1 Score': [adaboost_metrics['f1'], gb_metrics['f1'], xgb_metrics['f1'], xgb_cat_metrics['f1'], catboost_metrics['f1'], catboost_cat_metrics['f1']]
})

# Сортируем по убыванию F1 Score
metrics_comparison = metrics_comparison.sort_values('F1 Score', ascending=False).reset_index(drop=True)

# Выводим сравнение метрик
metrics_comparison

In [ ]:
# Определяем лучшую модель на основе F1 Score
best_model_name = metrics_comparison.iloc[0]['Модель']
print(f"Лучшая модель: {best_model_name}")

# Выбираем лучшую модель для дальнейшего анализа
if best_model_name == 'CatBoost (как есть)':
    best_model = catboost_cat
elif best_model_name == 'CatBoost (закодированные)':
    best_model = best_catboost
elif best_model_name == 'XGBoost (как есть)':
    best_model = xgb_cat
elif best_model_name == 'XGBoost (закодированные)':
    best_model = best_xgb
elif best_model_name == 'GradientBoostingClassifier':
    best_model = best_gb
else:  # AdaBoost
    best_model = best_adaboost

## 4. Важность признаков

### 4.1 С использованием `shap.TreeExplainer` получить $SHAP$-значения для лучшей модели

In [ ]:
# Определяем, какие данные использовать для SHAP в зависимости от лучшей модели
if 'как есть' in best_model_name:
    X_shap = X_test
    feature_names = X_test.columns
else:
    X_shap = X_test_encoded
    feature_names = encoded_features

# Создаем SHAP explainer
explainer = shap.TreeExplainer(best_model)

# Вычисляем SHAP значения
shap_values = explainer.shap_values(X_shap)

# Если shap_values - это список (для мультиклассовой классификации), берем первый класс
if isinstance(shap_values, list):
    print(f"SHAP значения для {len(shap_values)} классов")
    # Для визуализации будем использовать значения для первого класса
    shap_values_class0 = shap_values[0]
else:
    print("SHAP значения для бинарной классификации или регрессии")
    shap_values_class0 = shap_values

### 4.2 Построить график `shap.plots.force` для одного объекта выборки и для среза произвольного размера

In [ ]:
# Для одного объекта выборки
shap.plots.force(explainer.expected_value[0] if isinstance(explainer.expected_value, list) else explainer.expected_value, 
                 shap_values_class0[0], 
                 X_shap.iloc[0], 
                 feature_names=feature_names)

In [ ]:
# Для среза произвольного размера (первые 5 объектов)
shap.plots.force(explainer.expected_value[0] if isinstance(explainer.expected_value, list) else explainer.expected_value, 
                 shap_values_class0[:5], 
                 X_shap.iloc[:5], 
                 feature_names=feature_names)

### 4.3 Построить график `shap.plots.bar` для одного объекта выборки

In [ ]:
# Создаем объект Explanation для одного объекта
exp = shap.Explanation(values=shap_values_class0[0], 
                       base_values=explainer.expected_value[0] if isinstance(explainer.expected_value, list) else explainer.expected_value,
                       data=X_shap.iloc[0].values,
                       feature_names=feature_names)

# Строим bar plot для одного объекта
shap.plots.bar(exp)

### 4.4 Построить график `shap.plots.waterfall` для одного объекта выборки

In [ ]:
# Строим waterfall plot для одного объекта
shap.plots.waterfall(exp)

### 4.5 Построить 2 графика `shap.plots.scatter` для какого-нибудь признака

In [ ]:
# Выбираем индекс признака для анализа (например, первый числовой признак)
feature_idx = 0

# График 1: Раскраска относительно самого признака
shap.plots.scatter(shap_values_class0[:, feature_idx], color=X_shap.iloc[:, feature_idx], 
                   feature_names=[feature_names[feature_idx]])

In [ ]:
# График 2: Раскраска относительно другого признака (например, второго числового признака)
other_feature_idx = 1
shap.plots.scatter(shap_values_class0[:, feature_idx], color=X_shap.iloc[:, other_feature_idx], 
                   feature_names=[feature_names[feature_idx]], color_bar_label=feature_names[other_feature_idx])

### 4.6 Построить график `shap.plots.beeswarm` для всех признаков

In [ ]:
# Строим beeswarm plot для всех признаков
shap.plots.beeswarm(shap.Explanation(values=shap_values_class0, 
                                    base_values=explainer.expected_value[0] if isinstance(explainer.expected_value, list) else explainer.expected_value,
                                    data=X_shap.values,
                                    feature_names=feature_names))

### 4.7 Построить график `shap.plots.bar` для всех признаков

In [ ]:
# Строим bar plot для всех признаков (средние абсолютные значения SHAP)
shap.plots.bar(shap.Explanation(values=shap_values_class0, 
                               base_values=explainer.expected_value[0] if isinstance(explainer.expected_value, list) else explainer.expected_value,
                               data=X_shap.values,
                               feature_names=feature_names))

### 4.8 На основании двух последних графиков и/или используя `feature_importance` отфильровать признаки

In [ ]:
# Получаем важность признаков из SHAP значений
feature_importance = np.abs(shap_values_class0).mean(axis=0)

# Создаем датафрейм с важностью признаков
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

# Выводим важность признаков
importance_df.head(20)

In [ ]:
# Визуализируем важность признаков
plt.figure(figsize=(12, 8))
plt.barh(importance_df['Feature'][:15], importance_df['Importance'][:15])
plt.xlabel('Средняя абсолютная величина SHAP')
plt.ylabel('Признак')
plt.title('Топ-15 признаков по важности')
plt.gca().invert_yaxis()  # Инвертируем ось Y для отображения самых важных признаков сверху
plt.tight_layout()
plt.show()

In [ ]:
# Определяем порог важности (например, берем 50% самых важных признаков)
n_features_to_keep = len(feature_names) // 2
important_features = importance_df['Feature'][:n_features_to_keep].values

print(f"Количество отобранных признаков: {len(important_features)}")
print("Отобранные признаки:")
print(important_features)

### 4.9 Переобучить лучшую модель на отфильтрованных признаках

In [ ]:
# Фильтруем данные, оставляя только важные признаки
if 'как есть' in best_model_name:
    # Для моделей с категориальными признаками "как есть"
    important_indices = [list(X_train.columns).index(feat) for feat in important_features if feat in X_train.columns]
    X_train_filtered = X_train.iloc[:, important_indices]
    X_test_filtered = X_test.iloc[:, important_indices]
    
    # Обновляем индексы категориальных признаков
    cat_features_filtered = []
    for i, feat in enumerate(X_train_filtered.columns):
        if feat in categorical_cols:
            cat_features_filtered.append(i)
else:
    # Для моделей с закодированными признаками
    important_indices = [list(encoded_features).index(feat) for feat in important_features if feat in encoded_features]
    X_train_filtered = X_train_encoded[:, important_indices]
    X_test_filtered = X_test_encoded[:, important_indices]
    cat_features_filtered = []

print(f"Размер отфильтрованной обучающей выборки: {X_train_filtered.shape}")
print(f"Размер отфильтрованной тестовой выборки: {X_test_filtered.shape}")

In [ ]:
# Переобучаем лучшую модель на отфильтрованных признаках
if 'CatBoost (как есть)' in best_model_name:
    filtered_model = CatBoostClassifier(
        iterations=best_catboost.iterations,
        learning_rate=best_catboost.learning_rate,
        depth=best_catboost.depth,
        l2_leaf_reg=best_catboost.l2_leaf_reg,
        border_count=best_catboost.border_count,
        bagging_temperature=best_catboost.bagging_temperature,
        random_strength=best_catboost.random_strength,
        random_seed=42,
        verbose=False,
        cat_features=cat_features_filtered
    )
elif 'CatBoost (закодированные)' in best_model_name:
    filtered_model = CatBoostClassifier(
        iterations=best_catboost.iterations,
        learning_rate=best_catboost.learning_rate,
        depth=best_catboost.depth,
        l2_leaf_reg=best_catboost.l2_leaf_reg,
        border_count=best_catboost.border_count,
        bagging_temperature=best_catboost.bagging_temperature,
        random_strength=best_catboost.random_strength,
        random_seed=42,
        verbose=False
    )
elif 'XGBoost (как есть)' in best_model_name:
    filtered_model = XGBClassifier(
        n_estimators=best_xgb.n_estimators,
        learning_rate=best_xgb.learning_rate,
        max_depth=best_xgb.max_depth,
        subsample=best_xgb.subsample,
        colsample_bytree=best_xgb.colsample_bytree,
        reg_alpha=best_xgb.reg_alpha,
        reg_lambda=best_xgb.reg_lambda,
        random_state=42,
        enable_categorical=True
    )
elif 'XGBoost (закодированные)' in best_model_name:
    filtered_model = XGBClassifier(
        n_estimators=best_xgb.n_estimators,
        learning_rate=best_xgb.learning_rate,
        max_depth=best_xgb.max_depth,
        subsample=best_xgb.subsample,
        colsample_bytree=best_xgb.colsample_bytree,
        reg_alpha=best_xgb.reg_alpha,
        reg_lambda=best_xgb.reg_lambda,
        random_state=42
    )
elif 'GradientBoostingClassifier' in best_model_name:
    filtered_model = GradientBoostingClassifier(
        n_estimators=best_gb.n_estimators,
        learning_rate=best_gb.learning_rate,
        max_depth=best_gb.max_depth,
        subsample=best_gb.subsample,
        min_samples_split=best_gb.min_samples_split,
        random_state=42
    )
else:  # AdaBoost
    filtered_model = AdaBoostClassifier(
        n_estimators=best_adaboost.n_estimators,
        learning_rate=best_adaboost.learning_rate,
        algorithm=best_adaboost.algorithm,
        random_state=42
    )

# Обучаем модель на отфильтрованных признаках
filtered_model.fit(X_train_filtered, y_cls_train)

# Делаем предсказания на тестовой выборке
y_pred_filtered = filtered_model.predict(X_test_filtered)

### 4.10 Сравнить метрики до и после фильтрации

In [ ]:
# Вычисляем метрики для модели с отфильтрованными признаками
filtered_metrics = evaluate_classifier(y_cls_test, y_pred_filtered, f"{best_model_name} (с отфильтрованными признаками)")

In [ ]:
# Сравниваем метрики до и после фильтрации
comparison_df = pd.DataFrame({
    'Метрика': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'До фильтрации': [metrics_comparison.iloc[0]['Accuracy'], metrics_comparison.iloc[0]['Precision'], 
                     metrics_comparison.iloc[0]['Recall'], metrics_comparison.iloc[0]['F1 Score']],
    'После фильтрации': [filtered_metrics['accuracy'], filtered_metrics['precision'], 
                        filtered_metrics['recall'], filtered_metrics['f1']],
    'Изменение': [filtered_metrics['accuracy'] - metrics_comparison.iloc[0]['Accuracy'], 
                 filtered_metrics['precision'] - metrics_comparison.iloc[0]['Precision'],
                 filtered_metrics['recall'] - metrics_comparison.iloc[0]['Recall'],
                 filtered_metrics['f1'] - metrics_comparison.iloc[0]['F1 Score']]
})

comparison_df

### 4.11 Визуализировать полученное дерево решений

In [ ]:
# Визуализация дерева решений зависит от типа модели
if 'CatBoost' in best_model_name:
    # Для CatBoost можно визуализировать одно из деревьев
    tree_idx = 0  # Индекс дерева для визуализации
    filtered_model.plot_tree(tree_idx=tree_idx)
    plt.figure(figsize=(20, 10))
    plt.show()
elif 'XGBoost' in best_model_name:
    # Для XGBoost можно использовать plot_tree
    xgb.plot_tree(filtered_model, num_trees=0)
    plt.figure(figsize=(20, 10))
    plt.show()
elif 'GradientBoostingClassifier' in best_model_name:
    # Для GradientBoostingClassifier можно визуализировать одно из деревьев
    tree_idx = 0  # Индекс дерева для визуализации
    estimator = filtered_model.estimators_[tree_idx, 0]
    
    # Определяем имена признаков для визуализации
    if 'как есть' in best_model_name:
        feature_names_viz = X_train_filtered.columns
    else:
        feature_names_viz = important_features
    
    # Экспортируем дерево в формат DOT
    dot_data = export_graphviz(
        estimator,
        out_file=None,
        feature_names=feature_names_viz,
        filled=True,
        rounded=True,
        special_characters=True
    )
    
    # Визуализируем дерево
    graph = graphviz.Source(dot_data)
    graph

## 5. Регрессия

### 5.1 С использованием `GridSearchCV`/`grid_search` осуществить подбор гиперпараметра модели линейной регрессии с использованием $XGBoost$ или $CatBoost$

In [ ]:
# Определяем, какую модель использовать для регрессии (XGBoost или CatBoost)
# Выберем ту, которая показала лучшие результаты в классификации
if 'CatBoost' in best_model_name:
    # Определяем сетку параметров для CatBoostRegressor
    reg_params = {
        'iterations': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.3],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [1, 3, 5],
        'border_count': [32, 64],
        'bagging_temperature': [0, 1],
        'random_strength': [1, 10]
    }
    
    # Создаем объект CatBoostRegressor
    reg_model = CatBoostRegressor(
        random_seed=42,
        verbose=False,
        train_size=0.8,  # Часть обучающей выборки пойдет в валидационный набор
        early_stopping_rounds=10
    )
    
    model_name = "CatBoostRegressor"
else:
    # Определяем сетку параметров для XGBRegressor
    reg_params = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'reg_alpha': [0, 0.1, 1.0],
        'reg_lambda': [0, 0.1, 1.0]
    }
    
    # Создаем объект XGBRegressor
    reg_model = XGBRegressor(
        random_state=42,
        early_stopping_rounds=10
    )
    
    model_name = "XGBRegressor"

# Создаем объект GridSearchCV
reg_grid = GridSearchCV(reg_model, reg_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Обучаем модель с подбором параметров
if 'как есть' in best_model_name and model_name == "CatBoostRegressor":
    # Для CatBoost с категориальными признаками "как есть"
    reg_grid.fit(X_train, y_reg_train, cat_features=cat_features_indices)
else:
    # Для остальных случаев
    reg_grid.fit(X_train_encoded, y_reg_train)

# Выводим лучшие параметры
print(f"Лучшие параметры для {model_name}: {reg_grid.best_params_}")
print(f"Лучший score (отрицательный MSE): {reg_grid.best_score_:.2f}")

# Получаем лучшую модель
best_reg_model = reg_grid.best_estimator_

# Делаем предсказания на тестовой выборке
if 'как есть' in best_model_name and model_name == "CatBoostRegressor":
    y_pred_reg = best_reg_model.predict(X_test)
else:
    y_pred_reg = best_reg_model.predict(X_test_encoded)

### 5.2 Вывести метрики $MSE$, $MAE$ и $R^2$ на тестовом наборе

In [ ]:
# Функция для вычисления и вывода метрик регрессии
def evaluate_regressor(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"Метрики для модели {model_name}:")
    print(f"MSE: {mse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R^2: {r2:.4f}")
    
    return {'mse': mse, 'mae': mae, 'r2': r2}

# Вычисляем метрики для регрессионной модели
reg_metrics = evaluate_regressor(y_reg_test, y_pred_reg, model_name)

### 5.3 Переобучить модель на отфильтрованном наборе признаков из пункта 4

In [ ]:
# Переобучаем регрессионную модель на отфильтрованных признаках
if model_name == "CatBoostRegressor":
    reg_filtered_model = CatBoostRegressor(
        iterations=best_reg_model.iterations,
        learning_rate=best_reg_model.learning_rate,
        depth=best_reg_model.depth,
        l2_leaf_reg=best_reg_model.l2_leaf_reg,
        border_count=best_reg_model.border_count,
        bagging_temperature=best_reg_model.bagging_temperature,
        random_strength=best_reg_model.random_strength,
        random_seed=42,
        verbose=False,
        cat_features=cat_features_filtered if 'как есть' in best_model_name else None
    )
else:  # XGBRegressor
    reg_filtered_model = XGBRegressor(
        n_estimators=best_reg_model.n_estimators,
        learning_rate=best_reg_model.learning_rate,
        max_depth=best_reg_model.max_depth,
        subsample=best_reg_model.subsample,
        colsample_bytree=best_reg_model.colsample_bytree,
        reg_alpha=best_reg_model.reg_alpha,
        reg_lambda=best_reg_model.reg_lambda,
        random_state=42,
        enable_categorical=True if 'как есть' in best_model_name else False
    )

# Обучаем модель на отфильтрованных признаках
reg_filtered_model.fit(X_train_filtered, y_reg_train)

# Делаем предсказания на тестовой выборке
y_pred_reg_filtered = reg_filtered_model.predict(X_test_filtered)

### 5.4 Сравнить метрики до и после фильтрации

In [ ]:
# Вычисляем метрики для регрессионной модели с отфильтрованными признаками
reg_filtered_metrics = evaluate_regressor(y_reg_test, y_pred_reg_filtered, f"{model_name} (с отфильтрованными признаками)")

In [ ]:
# Сравниваем метрики до и после фильтрации
reg_comparison_df = pd.DataFrame({
    'Метрика': ['MSE', 'MAE', 'R^2'],
    'До фильтрации': [reg_metrics['mse'], reg_metrics['mae'], reg_metrics['r2']],
    'После фильтрации': [reg_filtered_metrics['mse'], reg_filtered_metrics['mae'], reg_filtered_metrics['r2']],
    'Изменение': [reg_filtered_metrics['mse'] - reg_metrics['mse'], 
                 reg_filtered_metrics['mae'] - reg_metrics['mae'],
                 reg_filtered_metrics['r2'] - reg_metrics['r2']]
})

reg_comparison_df